In [3]:
import re
import pandas as pd
from nltk.corpus import stopwords
import fasttext

# Data

In [4]:
data = pd.read_csv('data/input/perfumery.zip')
data.dropna(inplace=True)
data.head()

,comment_text
0,"Сразу скажу, что аромат на любителя. Но меня о..."
1,"когда мне дали послушать этот аромат, мне так ..."
2,Замечательная пара к женской новинке этой марк...
3,Отличный фруктовый аромат!🍊
4,"Очень классный набор, отлично подойдёт на пода..."


### Data cleanup

In [5]:
regex = re.compile("[А-ЯЁа-яёA-z]+")
mystopwords = stopwords.words(fileids='russian')

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 1]

def clean_text(text):
    tokens = words_only(text)
    return ' '.join(remove_stopwords(tokens))

### Saving a set for model training

In [68]:
with open("p.txt", "w", encoding='UTF-8') as file:
    for line in data.comment_text.values:
        if len(line) > 300:
            file.write(clean_text(line) + '\n')

# Model 

In [14]:
# original BIN model loading
model = fasttext.load_model("model/adaptation/perfumery.bin")
lines = []

# get all words from model
words = model.get_words()

with open('model.vec','w') as file_out:
    
    # the first line must contain number of total words and vector dimension
    file_out.write(str(len(words)) + " " + str(model.get_dimension()) + "\n")

    # line by line, you append vectors to VEC file
    for w in words:
        v = model.get_word_vector(w)
        vstr = ""
        for vi in v:
            vstr += " " + str(vi)
        try:
            file_out.write(w + vstr+'\n')
        except:
            pass

In [7]:
import fasttext

In [5]:
model = fasttext.load_model("model/adaptation/perfumery_epoch_1.bin")

In [8]:
print("building vocabulary...")
model.build_vocab('perfumery.txt', update=True)

building vocabulary...


model

In [10]:
model.wv.most_similar('парфюм')

[('парфюмы', 0.7377110123634338),
 ('аромат', 0.7371137142181396),
 ('парфюмерный', 0.7119106650352478),
 ('Парфюм', 0.7044569849967957),
 ('парфюм.', 0.6883740425109863),
 ('парфюма', 0.6814783811569214),
 ('духи', 0.6683558821678162),
 ('дезодорант', 0.6603888273239136),
 ('парфюмов', 0.6553675532341003),
 ('парфум', 0.6519331336021423)]

In [11]:
model.wv.most_similar('фуруктовый')

[('Гуманитарно-экономический', 0.561374306678772),
 ('магистратурыучебно-методический', 0.5601953268051147),
 ('Врачебно-физкультурный', 0.5565395355224609),
 ('аграрно-технологический', 0.5547544360160828),
 ('ревматологический', 0.5517967343330383),
 ('Гуманитарно-педагогический', 0.5509157180786133),
 ('кардиоревматологический', 0.5496220588684082),
 ('пектиняблочный', 0.5401729941368103),
 ('аграрно-экономический', 0.5355527997016907),
 ('историко-культурологический', 0.5343062877655029)]